<a href="https://colab.research.google.com/github/GoetheUniCV/cvproject/blob/main/Luis/covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Grad-CAM for COVID-19

##1. Setup
###1.1 Python Environment
Almost all required packages are preinstalled, except _kerastuner_. You can install it with `!pip install -U keras-tuner`.

###1.2 Data
Upload the COVID-19 x-ray data to your personal Google Drive folder.

___


In [ ]:
!pip install -U keras-tuner

In [7]:
#import librarires
import numpy as np
import pandas as pd

import kerastuner as kt
from tensorflow import keras
import tensorflow as tf
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter as hp
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import VGG19
from keras import layers
from keras.preprocessing import image
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

In [8]:
# This mounts your Google Drive to the Colab VM.
drive.mount('/content/drive', force_remount=True)

# Foldername in your Drive where you have saved the COVID-19 data.
FOLDERNAME = 'COVID-19 Radiography Database'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


**Hint**: You can run terminal commands in your notebook if you prefix them with `%`:

In [4]:
# Go to the directory with the COVID-19 data.
%cd drive/My\ Drive/$FOLDERNAME
# List its content.
%ls

[Errno 2] No such file or directory: 'drive/My Drive/COVID-19 Radiography Database'
/content/drive/My Drive/COVID-19 Radiography Database
 COVID-19/                NORMAL.metadata.xlsx  'Viral Pneumonia.matadata.xlsx'
 COVID-19.metadata.xlsx   README.md.txt
 NORMAL/                 'Viral Pneumonia'/


In [3]:
%ls

 COVID-19/                NORMAL.metadata.xlsx  'Viral Pneumonia.matadata.xlsx'
 COVID-19.metadata.xlsx   README.md.txt
 NORMAL/                 'Viral Pneumonia'/
